## Contectando e pesquisando valores no SQLServer 19

https://planetscale.com/blog/using-mysql-with-sql-alchemy-hands-on-examples

### Modules

In [ ]:
import sqlalchemy as db
from sqlalchemy import create_engine, text
import pyodbc
import io
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

connection_uri = db.engine.URL.create(
    "mssql+pyodbc",
    username="sa",
    password="Dash@0130",
    host="172.18.144.1,1433",
    database="DataDocAI",
    query={
        "driver": "ODBC Driver 17 for SQL Server",
    },
)
engine = create_engine(connection_uri)
connection = engine.connect()

In [ ]:
index = 8


result = connection.execute(text(f"Select * from  LAI_LEIS_MUNICIPAIS where LEI_CODIGO = {index}"))
for row in result.mappings():
    lei_row = row
    lei_codigo = row["LEI_CODIGO"]
    lei_descricao =  row["LEI_DESCRICAO"]
    lei_data_upload = row["LEI_DATA_UPLOAD"]
    lei_guid = row["LEI_GUID"]
    lei_nome_arquivo = row["LEI_NOME_ARQUIVO"]
    lei_ano = row["LEI_ANO"]
    lei_decricao_numero = row["LEI_DESCRICAO_NUMERO"]
    lei_url = row["URL_DOWNLOAD"]
    lei_data = row["LEI_DATA"]
    lei_data_ultima = row["data_ultima"]
    lei_arquivo = row["LEI_ARQUIVO"]
    file_data = row["LEI_ARQUIVO"]
    
    print(f'data: {lei_data} | Descricao: {lei_descricao} | url: {lei_url}')
    
    
if file_data:
    # Crie um arquivo PDF a partir do valor binário.
    with open('data/output.pdf', 'wb') as pdf_file:
        pdf_file.write(file_data)

    print('Arquivo PDF criado com sucesso!')
else:
    print('O valor binário é nulo.')

### Tratando o PDF

Portable Document Format (PDF), padronizado como ISO 32000, é um formato de arquivo desenvolvido pela Adobe em 1992 para apresentar documentos, incluindo formatação de texto e imagens, de maneira independente do software aplicativo, hardware e sistemas operacionais.

Isso aborda como carregar documentos PDF no formato de documento que usamos posteriormente.

In [ ]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("data/output.pdf")
pages = loader.load_and_split()
pages[1]

In [ ]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings())

In [ ]:
docs = faiss_index.similarity_search("como será realizada a fiscalização do contrato?", k=2)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:300])

### Using Unstructured

In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader

In [ ]:
loader = UnstructuredPDFLoader("data/output.pdf")

In [ ]:
data = loader.load()

### Retain Elements

Under the hood, Unstructured creates different "elements" for different chunks of text. By default we combine those together, but you can easily keep that separation by specifying mode="elements".

## Using PDFMiner to generate HTML text

This can be helpful for chunking texts semantically into sections as the output html content can be parsed via BeautifulSoup to get more structured and rich information about font size, page numbers, PDF headers/footers, etc.

In [ ]:
from langchain.document_loaders import PDFMinerPDFasHTMLLoader
loader = PDFMinerPDFasHTMLLoader("data/output.pdf")
data = loader.load()[0]   # entire PDF is loaded as a single Document

from bs4 import BeautifulSoup
soup = BeautifulSoup(data.page_content,'html.parser')
content = soup.find_all('div')

In [ ]:
soup

In [ ]:
import re
cur_fs = None
cur_text = ''
snippets = []   # first collect all snippets that have the same font size
for c in content:
    sp = c.find('span')
    #print(f'\nsp: {sp}')
    if not sp:
        continue
    st = sp.get('style')
    #print(f'\nst: {st}')
    if not st:
        continue
    fs = re.findall('font-size:(\d+)px',st)
    #print(f'\nfs: {fs}')
    if not fs:
        continue
    fs = int(fs[0])
    if not cur_fs:
        cur_fs = fs
        #print(f'\n     cur_fs: {cur_fs}')
    if fs == cur_fs:
        cur_text += c.text
        #print(f'\n cur_text: {cur_text}')
    else:
        snippets.append((cur_text,cur_fs))
        cur_fs = fs
        cur_text = c.text
snippets.append((cur_text,cur_fs))
# Nota: A lógica acima é muito direta. Também é possível adicionar mais estratégias, como remover trechos duplicados (como
# cabeçalhos/rodapés em um PDF aparecem em várias páginas, portanto, se encontrarmos duplicatas, é seguro assumir que são informações redundantes)

In [ ]:
# Isto e uma lista
snippets

In [ ]:
from langchain.docstore.document import Document
cur_idx = -1
semantic_snippets = []
# Suposição: os títulos têm tamanho de fonte maior que o respectivo conteúdo
for s in snippets:
    # se o tamanho da fonte do snippet atual > título da seção anterior => é um novo título
    if not semantic_snippets or s[1] > semantic_snippets[cur_idx].metadata['heading_font']:
        metadata={'heading':s[0], 'content_font': 0, 'heading_font': s[1]}
        metadata.update(data.metadata)
        semantic_snippets.append(Document(page_content='',metadata=metadata))
        cur_idx += 1
        continue

    # se o tamanho da fonte do snippet atual <= conteúdo da seção anterior => o conteúdo pertence à mesma seção (também é possível criar
    # uma estrutura em forma de árvore para subseções, se necessário, mas isso pode exigir um pouco mais de reflexão e pode ser específico dos dados)
    if not semantic_snippets[cur_idx].metadata['content_font'] or s[1] <= semantic_snippets[cur_idx].metadata['content_font']:
        semantic_snippets[cur_idx].page_content += s[0]
        semantic_snippets[cur_idx].metadata['content_font'] = max(s[1], semantic_snippets[cur_idx].metadata['content_font'])
        continue

    # se o tamanho da fonte do snippet atual > o conteúdo da seção anterior, mas menor que o título da seção anterior, faça também uma nova
    # seção (por exemplo, o título de um PDF terá o tamanho de fonte mais alto, mas não queremos que ele inclua todas as seções)
    metadata={'heading':s[0], 'content_font': 0, 'heading_font': s[1]}
    metadata.update(data.metadata)
    semantic_snippets.append(Document(page_content='',metadata=metadata))
    cur_idx += 1

In [ ]:
len(semantic_snippets)

In [ ]:
semantic_snippets[0]

In [ ]:
semantic_snippets[1]

In [ ]:
semantic_snippets[1].page_content

In [ ]:
semantic_snippets[2]

In [ ]:
semantic_snippets[3]

In [ ]:
texto = semantic_snippets[3]

In [ ]:
texto = semantic_snippets[1]

In [ ]:
type(texto)

In [ ]:
texto

In [ ]:
text_splited = texto.split('\n')

In [ ]:
text_splited

In [ ]:
text_splited = [x for x in text_splited if x.strip()]

In [ ]:
text_splited

In [ ]:
# Adiciona uma linha em branco entre cada elemento da lista
lista_com_linhas_em_branco = [item for sublist in zip(text_splited, [''] * len(text_splited)) for item in sublist]


In [ ]:
# Converte a lista para uma única string, com cada elemento separado por uma quebra de linha
texto_final = '\n'.join(lista_com_linhas_em_branco)

In [ ]:
texto_final

In [ ]:
# Abre o arquivo no modo de escrita ('w') e salva o texto
with open('data/doc_pdf.txt', 'w') as arquivo:
    arquivo.write(texto_final)

In [ ]:
texto_PDF_P = re.sub('\s+', ' ', text_P).strip()

## Document transformers

Depois de carregar os documentos, muitas vezes você desejará transformá-los para melhor atender à sua aplicação. O exemplo mais simples é que você pode querer dividir um documento longo em partes menores que caibam na janela de contexto do seu modelo. LangChain possui vários transformadores de documentos integrados que facilitam a divisão, combinação, filtragem e manipulação de documentos.

### Text splitters

In [ ]:
# This is a long document we can split up.
with open('data/state_of_the_union.txt') as f:
    state_of_the_union = f.read()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 100,
    chunk_overlap  = 40,
    length_function = len,
    add_start_index = True,
)

In [ ]:
document_to_split = semantic_snippets[4]

In [ ]:
texts = text_splitter.create_documents([state_of_the_union])
print(texts[0])
print(texts[1])
print(texts[3])

In [ ]:
ocr_count = 0
for page in doc:
    blocks = page.get_text("dict", flags=0)["blocks"]
    for b in blocks:
        for l in b["lines"]:
            for s in l["spans"]:
                text = s["text"]
                if chr(0x0024) in text:  # invalid characters encountered!    0xfffd
                    # invoke OCR
                    ocr_count += 1
                    new_text = get_tessocr(page, s)

print("-------------------------")
print("OCR invocations: %i." % ocr_count)
print(
    "Pixmap time: %g (avg %g) seconds."
    % (round(PIX_TIME, 5), round(PIX_TIME / ocr_count, 5))
)
print(
    "OCR time: %g (avg %g) seconds."
    % (round(OCR_TIME, 5), round(OCR_TIME / ocr_count, 5))
)

## HTMLHeaderTextSplitter

Semelhante em conceito ao MarkdownHeaderTextSplitter, o HTMLHeaderTextSplitter é um chunker "com reconhecimento de estrutura" que divide o texto no nível do elemento e adiciona metadados para cada cabeçalho "relevante" a qualquer pedaço. Ele pode retornar pedaços elemento por elemento ou combinar elementos com os mesmos metadados, com os objetivos de (a) manter o texto relacionado agrupado (mais ou menos) semanticamente e (b) preservar informações ricas em contexto codificadas em estruturas de documentos. Ele pode ser usado com outros divisores de texto como parte de um pipeline de chunking.

### Usage examples

In [9]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

In [ ]:
from langchain.text_splitter import HTMLHeaderTextSplitter

In [ ]:
html_string ="""
<!DOCTYPE html>
<html>
<body>
    <div>
        <h1>Foo</h1>
        <p>Some intro text about Foo.</p>
        <div>
            <h2>Bar main section</h2>
            <p>Some intro text about Bar.</p>
            <h3>Bar subsection 1</h3>
            <p>Some text about the first subtopic of Bar.</p>
            <h3>Bar subsection 2</h3>
            <p>Some text about the second subtopic of Bar.</p>
        </div>
        <div>
            <h2>Baz</h2>
            <p>Some text about Baz</p>
        </div>
        <br>
        <p>Some concluding text about Foo</p>
    </div>
</body>
</html>
"""

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
html_header_splits = html_splitter.split_text(html_string)
html_header_splits

In [ ]:
def texto_extraido(texto):
    #0. Tratamento da string
    text_splited = texto.split('\n')
    text_splited = [s.replace(":", "") for s in text_splited]
    text_splited = [x for x in text_splited if x.strip()]
    text_splited = [s.replace(";", "").strip() for s in text_splited] #depende da situaçao
    return text_splited

In [ ]:
text_splited = [s.replace("'", "") for s in text_splited]
text_splited = [s.replace("',", "").strip() for s in text_splited] #depende da situaçao
text_splited = texto.split('\n')